In [1]:
## AN AUDIOVISUAL ODE TO TITLE FIGHT
## USING SPOTIFY DATA TO FIGURE OUT WHY TITLE FIGHT IS A SCENE FAVORITE YEARS AFTER THEIR LAST ALBUM RELEASE
## BY SAWYER CLICK

#imports
import requests
import pandas as pd
import re
from dotenv import load_dotenv
load_dotenv()
import base64
import os
import requests
import json
pd.set_option('display.max_colwidth', 999)
# import nltk
# nltk.download()

# get tokens
spotify_client_id = os.getenv("tfID")
spotify_client_secret = os.getenv("tfSecret")
token = os.getenv("SPOTIFY_ACCESS_TOKEN")
genius_api_key = os.getenv("GENIUS_ACCESS")

# set up how we use the tokens
access_token = os.getenv("SPOTIFY_ACCESS_TOKEN")
raw_token = !spotify-token
token = f"Bearer {raw_token[1].replace('Your token is:  ','')}"
base_url = "https://api.spotify.com/v1"
headers = {'Authorization': token}

In [2]:
# grab artist data for title fight
tfURI = '2CnhqfjUG0qzsru0SMuhrk'
search_url = f"{base_url}/artists/{tfURI}"
params = {'limit':1}
artist = requests.get(search_url, params=params, headers=headers).json()

with open('artist.json', 'w') as outfile:
    json.dump(artist, outfile)
    
print('ARTIST DATA', artist)

ARTIST DATA {'external_urls': {'spotify': 'https://open.spotify.com/artist/2CnhqfjUG0qzsru0SMuhrk'}, 'followers': {'href': None, 'total': 148765}, 'genres': ['alternative emo', 'emo', 'indie punk', 'melodic hardcore', 'pop punk'], 'href': 'https://api.spotify.com/v1/artists/2CnhqfjUG0qzsru0SMuhrk', 'id': '2CnhqfjUG0qzsru0SMuhrk', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/f655f973e8e6ac3b8e556d1dbc11aac05d19a2b4', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/e00b58d31d4ef03ae029ae8f18005f01ea774ea6', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/5a23ff7df5d262390ae72dd9a3c9f58eaf943144', 'width': 160}], 'name': 'Title Fight', 'popularity': 53, 'type': 'artist', 'uri': 'spotify:artist:2CnhqfjUG0qzsru0SMuhrk'}


In [3]:
# getting album data
search_url = f"{base_url}/artists/{tfURI}/albums"
params = {'limit':50, 'include_groups': 'album,single'}
raw_albums = requests.get(search_url, params=params, headers=headers).json()

# limiting to US market to remove duplicate albums with one-off markets
albumsBase = []
for album in raw_albums['items']:
    if 'US' in album['available_markets']:
        del album['available_markets']
        albumsBase.append(album)

with open('albumsBase.json', 'w') as outfile:
    json.dump(albumsBase, outfile)
    
print('Collecting uris for these albums & singles:', [album['name'] for album in albumsBase])  

In [4]:
baseSearchAlbum = f'{base_url}/albums/'
albums = []
tracksBase = []
for album in albumsBase:
    
    #store data from this album in our albums array
    thisAlbum = requests.get(baseSearchAlbum+album['uri'].split(':')[2], params={'limit':1}, headers=headers).json()
    del thisAlbum['available_markets']
    albums.append(thisAlbum)
    
    # create a function to grab all the tracks and get their base data
    # eventually add endpoints for collecting anaylsis
    albumTracks = requests.get(thisAlbum['tracks']['href'], headers=headers).json()
    for track in albumTracks['items']:
        # attach the album name
        track['album'] = album['name']
        # delete the available markets array bc it takes up so much space
        del track['available_markets']
        tracksBase.append(track)
        
trackURIs = [track['uri'].split(':')[2] for track in tracksBase]
        
with open('albums.json', 'w') as outfile:
    json.dump(albums, outfile)

In [5]:
# grab the features of each track in their albums
tracksFt = []
[tracksFt.append(requests.get(base_url+'/audio-features/'+uri, headers=headers).json()) for uri in trackURIs]
print(f'Grabbed data for {len(trackURIs)} tracks')

Grabbed data for 51 tracks


In [6]:
for track in tracksBase:
    for ftTrack in tracksFt:
        if track['uri'] == ftTrack['uri']:
            track['features'] = ftTrack

In [7]:
with open('tracks.json', 'w') as outfile:
    json.dump(tracksBase, outfile)